In [124]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Display the full content of each column
import re
from tqdm import tqdm
tqdm.pandas()  # This adds the tqdm progress bar to pandas operations
from openai import OpenAI
client = OpenAI()
from dotenv import load_dotenv
import json
ret = load_dotenv()
import ast

In [125]:
df1 = pd.read_csv("../data/Acura RAW Data CSV Report - Sheet1.csv")
df2 = pd.read_csv("../data/Raw Data Acura - Including Rebates - Sheet1.csv")

In [126]:
df1.sample(2)

,Package,Rate Model,Bank,MSRP,Invoice,Formula,Buying,Selling Price,Default Rebates,Nondefault Rebates,Rate Style,MF 24,RP 24,MF 27,RP 27,MF 30,RP 30,MF 33,RP 33,MF 36,RP 36,MF 39,RP 39,MF 42,RP 42,MF 48,RP 48,Package Id,Bank Id
48,2024 Acura MDX AWD 7 Passenger,MDX AWD,Cal Automotive,75750,71140,0.939142,0,71140,NaN,NaN,YD8H8RKNW 4dr Wgn Type-S Advance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0032,59,0.0032,59,NaN,NaN,NaN,NaN,65538,48
29,2024 Acura RDX AWD SUV,RDX AWD,American Honda Finance (Acura),51495,48972,0.951005,-1800,50772,NaN,NaN,TC2H6RKNW 4dr Wgn A-Spec AWD,0.0038,56.0,0.0038,55.0,0.0038,53.0,0.0028,51.0,0.0028,49,0.0028,47,0.00295,45.0,0.00295,41.0,65533,3


In [127]:
df2.head(2)

,Search:,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Package,Bank,MSRP,Invoice,Formula,Buying,Selling Price,MF,Residual,Default Rebates,Adjusted Cap Cost,Non Default Rebates,Rate Model,Rate Style,Term
1,2024 Acura ZDX AWD SUV,American Honda Finance (Acura),"70,450","69,208",98.24%,0,"69,208",0.0012,41%,"Clean Vehicle Lease Credit (7,500)\nCaptive Le...",NaN,Acura Military Appreciation Offer (750)\nAcura...,ZDX AWD,3B2H4REW 4dr Wgn A-Spec,36


In [128]:
df2.columns = df2.iloc[0]  # Assign first row as column names
df2 = df2[1:].reset_index(drop=True)  # Drop the first row and reset index


In [129]:
df2.head(2)

,Package,Bank,MSRP,Invoice,Formula,Buying,Selling Price,MF,Residual,Default Rebates,Adjusted Cap Cost,Non Default Rebates,Rate Model,Rate Style,Term
0,2024 Acura ZDX AWD SUV,American Honda Finance (Acura),"70,450","69,208",98.24%,0,"69,208",0.0012,41%,"Clean Vehicle Lease Credit (7,500)\nCaptive Le...",NaN,Acura Military Appreciation Offer (750)\nAcura...,ZDX AWD,3B2H4REW 4dr Wgn A-Spec,36
1,2024 Acura ZDX AWD SUV,American Honda Finance (Acura),"75,450","73,970",98.04%,0,"73,970",0.0012,40%,"Clean Vehicle Lease Credit (7,500)\nCaptive Le...",NaN,Acura Military Appreciation Offer (750)\nAcura...,ZDX AWD,3B2H6REW 4dr Wgn Type S,36


In [130]:
df2.columns

Index(['Package', 'Bank', 'MSRP', 'Invoice', 'Formula', 'Buying',
       'Selling Price', 'MF', 'Residual', 'Default Rebates',
       'Adjusted Cap Cost', 'Non Default Rebates', 'Rate Model', 'Rate Style',
       'Term'],
      dtype='object', name=0)

In [131]:
df2 = df2[['Package', 'Bank', 'MSRP', 
       'Selling Price', 'Default Rebates',
       'Adjusted Cap Cost']]

In [132]:
# Regular expression to extract the total
pattern = r'Total:\s*([\d,]+)'

# Function to extract total value
def extract_total(text):
    match = re.search(pattern, text)
    return int(match.group(1).replace(',', '')) if match else None

# Create a new column in df2
df2["Total_Rebates"] = df2["Default Rebates"].apply(extract_total)


In [133]:
# Remove commas and convert to float
df2["Selling Price"] = df2["Selling Price"].str.replace(",", "").astype(int)
df2["MSRP"] = df2["MSRP"].str.replace(",", "").astype(int)
df2["Total_Rebates"] = df2["Total_Rebates"].astype(int)  # Ensure this is also float

df2["Adjusted Cap Cost"] = df2["Selling Price"]-df2['Total_Rebates']

In [134]:
df2

,Package,Bank,MSRP,Selling Price,Default Rebates,Adjusted Cap Cost,Total_Rebates
0,2024 Acura ZDX AWD SUV,American Honda Finance (Acura),70450,69208,"Clean Vehicle Lease Credit (7,500)\nCaptive Le...",47358,21850
1,2024 Acura ZDX AWD SUV,American Honda Finance (Acura),75450,73970,"Clean Vehicle Lease Credit (7,500)\nCaptive Le...",55620,18350
2,2025 Acura Integra Hatchback,American Honda Finance (Acura),39195,37689,Captive Lease Offer (350)\nAcura Lease Cash (0...,37339,350
3,2025 Acura Integra Hatchback,American Honda Finance (Acura),39195,37689,Captive Lease Offer (350)\nAcura Lease Cash (0...,37339,350
4,2025 Acura Integra Hatchback,American Honda Finance (Acura),34195,33377,Captive Lease Offer (350)\nAcura Lease Cash (0...,33027,350
5,2025 Acura Integra Hatchback,American Honda Finance (Acura),36195,34864,Captive Lease Offer (350)\nAcura Lease Cash (0...,34514,350
6,2025 Acura Integra Hatchback,American Honda Finance (Acura),54695,52594,Captive Lease Offer (350)\nAcura Lease Cash (0...,52244,350
7,2025 Acura MDX 2WD 7 Passenger,American Honda Finance (Acura),58250,56130,"Captive Lease Offer (2,900)\nAcura Lease Cash ...",53230,2900
8,2025 Acura MDX AWD 7 Passenger,American Honda Finance (Acura),63450,61226,"Captive Lease Offer (2,900)\nAcura Lease Cash ...",58326,2900
9,2025 Acura MDX AWD 7 Passenger,American Honda Finance (Acura),60450,58523,"Captive Lease Offer (2,900)\nAcura Lease Cash ...",55623,2900


In [135]:
df = pd.merge(df1,df2,on = ["Package","Bank","MSRP"],suffixes=("","_36"))

In [136]:
df.columns

Index(['Package', 'Rate Model', 'Bank', 'MSRP', 'Invoice', 'Formula', 'Buying',
       'Selling Price', 'Default Rebates', 'Nondefault Rebates', 'Rate Style',
       'MF 24', 'RP 24', 'MF 27', 'RP 27', 'MF 30', 'RP 30', 'MF 33', 'RP 33',
       'MF 36', 'RP 36', 'MF 39', 'RP 39', 'MF 42', 'RP 42', 'MF 48', 'RP 48',
       'Package Id', 'Bank Id', 'Selling Price_36', 'Default Rebates_36',
       'Adjusted Cap Cost', 'Total_Rebates'],
      dtype='object')

In [137]:
df1.shape

(66, 29)

In [138]:
df2.shape

(39, 7)

In [139]:
df.shape

(43, 33)

In [140]:
df.drop_duplicates(inplace=True)

In [141]:
df[['Package', 'Rate Model', 'Bank', 'MSRP', 'Invoice', 'Formula', 'Buying','Selling Price_36', 
       'Selling Price', 'Default Rebates', 'Nondefault Rebates','Default Rebates_36','Total_Rebates', 
       'Adjusted Cap Cost', 'Rate Style',
       'MF 24', 'RP 24', 'MF 27', 'RP 27', 'MF 30', 'RP 30', 'MF 33', 'RP 33',
       'MF 36', 'RP 36', 'MF 39', 'RP 39', 'MF 42', 'RP 42', 'MF 48', 'RP 48',
       'Package Id', 'Bank Id', ]].to_csv("../data/merged_data.csv",index=False)

In [142]:
df = df[['Package', 'Rate Model', 'Bank', 'MSRP', 'Invoice', 'Formula', 'Buying','Selling Price_36', 
       'Selling Price', 'Default Rebates', 'Nondefault Rebates','Default Rebates_36','Total_Rebates', 
       'Adjusted Cap Cost', 'Rate Style',
       'MF 24', 'RP 24', 'MF 27', 'RP 27', 'MF 30', 'RP 30', 'MF 33', 'RP 33',
       'MF 36', 'RP 36', 'MF 39', 'RP 39', 'MF 42', 'RP 42', 'MF 48', 'RP 48',
       'Package Id', 'Bank Id', ]]

In [144]:
def car_details_extraction(car):
    try:

        response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
            {
              "role": "system",
              "content": [
                {
                  "type": "text",
                  "text": "You are a car expert. you will be given a car details, extract the following as json\n\nYear\nMake\nModel\nTrim "
                }
              ]
            },
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": "2024 Acura ZDX AWD SUV"
                }
              ]
            },
            {
              "role": "assistant",
              "content": [
                {
                  "type": "text",
                  "text": "{\n  \"Year\": 2024,\n  \"Make\": \"Acura\",\n  \"Model\": \"ZDX\",\n  \"Trim\": \"AWD SUV\"\n}"
                }
              ]
            },
              {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": car
                }
              ]}

          ],
          response_format={
            "type": "json_object"
          },
          temperature=0,
          max_completion_tokens=2048,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        return response.choices[0].message.content
        
    except:
        print(car)
        return None

In [145]:
df["car_details"] = df.Package.progress_apply(car_details_extraction)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:53<00:00,  1.38s/it]


In [146]:

df['Year'] = df['car_details'].apply(
            lambda x: json.loads(x)["Year"] if isinstance(x, str) else pd.NA
        )
df['Make'] = df['car_details'].apply(
            lambda x: json.loads(x)["Make"] if isinstance(x, str) else pd.NA
        )
df['Model'] = df['car_details'].apply(
            lambda x: json.loads(x)["Model"] if isinstance(x, str) else pd.NA
        )
df['Trim'] = df['car_details'].apply(
            lambda x: json.loads(x)["Trim"] if isinstance(x, str) else pd.NA
        )

In [147]:
df.to_csv("../data/merged_data_v2.csv")

In [148]:
df

,Package,Rate Model,Bank,MSRP,Invoice,Formula,Buying,Selling Price_36,Selling Price,Default Rebates,Nondefault Rebates,Default Rebates_36,Total_Rebates,Adjusted Cap Cost,Rate Style,MF 24,RP 24,MF 27,RP 27,MF 30,RP 30,MF 33,RP 33,MF 36,RP 36,MF 39,RP 39,MF 42,RP 42,MF 48,RP 48,Package Id,Bank Id,car_details,Year,Make,Model,Trim
0,2024 Acura ZDX AWD SUV,ZDX AWD,American Honda Finance (Acura),70450,69208,0.982370,0,69208,69208,NaN,NaN,"Clean Vehicle Lease Credit (7,500)\nCaptive Le...",21850,47358,3B2H4REW 4dr Wgn A-Spec,0.00120,48.0,0.00120,47.0,0.00120,45.0,0.00120,43.0,0.00120,41,0.00280,39,0.00295,36.0,0.00295,31.0,70563,3,"{\n ""Year"": 2024,\n ""Make"": ""Acura"",\n ""Mod...",2024,Acura,ZDX,AWD SUV
1,2024 Acura ZDX AWD SUV,ZDX AWD,American Honda Finance (Acura),75450,73970,0.980384,0,73970,73970,NaN,NaN,"Clean Vehicle Lease Credit (7,500)\nCaptive Le...",18350,55620,3B2H6REW 4dr Wgn Type S,0.00120,47.0,0.00120,46.0,0.00120,44.0,0.00120,42.0,0.00120,40,0.00280,38,0.00295,35.0,0.00295,30.0,70564,3,"{\n ""Year"": 2024,\n ""Make"": ""Acura"",\n ""Mod...",2024,Acura,ZDX,AWD SUV
2,2024 Acura Integra Hatchback,Integra,American Honda Finance (Acura),37695,36241,0.961427,-600,36841,36841,NaN,NaN,Acura Lease Cash (0)\nTotal: 0\nlink,0,36841,DE4G7RJW 5dr HB A-Spec Tech Package MT,0.00380,58.0,0.00380,57.0,0.00380,55.0,0.00280,53.0,0.00280,51,0.00280,49,0.00295,46.0,0.00295,41.0,59914,3,"{\n ""Year"": 2024,\n ""Make"": ""Acura"",\n ""Mod...",2024,Acura,Integra,Hatchback
4,2025 Acura Integra Hatchback,Integra,American Honda Finance (Acura),39195,37689,0.961577,0,37689,37689,NaN,NaN,Captive Lease Offer (350)\nAcura Lease Cash (0...,350,37339,DE4G7SJW 5dr HB A-Spec Tech Package MT,0.00180,62.0,0.00180,61.0,0.00180,59.0,0.00180,57.0,0.00180,55,0.00180,53,0.00180,51.0,0.00180,47.0,72036,3,"{\n ""Year"": 2025,\n ""Make"": ""Acura"",\n ""Mod...",2025,Acura,Integra,Hatchback
6,2024 Acura Integra Hatchback,Integra,American Honda Finance (Acura),32695,31907,0.975898,-200,32107,32107,NaN,NaN,Acura Lease Cash (0)\nTotal: 0\nlink,0,32107,DE4H2RJW 5dr HB AT,0.00380,60.0,0.00380,59.0,0.00380,57.0,0.00280,55.0,0.00280,53,0.00280,51,0.00295,48.0,0.00295,43.0,59911,3,"{\n ""Year"": 2024,\n ""Make"": ""Acura"",\n ""Mod...",2024,Acura,Integra,Hatchback
7,2025 Acura Integra Hatchback,Integra,American Honda Finance (Acura),34195,33377,0.976078,0,33377,33377,NaN,NaN,Captive Lease Offer (350)\nAcura Lease Cash (0...,350,33027,DE4H2SJW 5dr HB AT,0.00180,63.0,0.00180,62.0,0.00180,60.0,0.00180,58.0,0.00180,56,0.00180,54,0.00180,52.0,0.00180,48.0,72033,3,"{\n ""Year"": 2025,\n ""Make"": ""Acura"",\n ""Mod...",2025,Acura,Integra,Hatchback
8,2024 Acura Integra Hatchback,Integra,American Honda Finance (Acura),34695,33366,0.961695,-200,33566,33566,NaN,NaN,Acura Lease Cash (0)\nTotal: 0\nlink,0,33566,DE4H3RJW 5dr HB A-Spec Package AT,0.00380,58.0,0.00380,57.0,0.00380,55.0,0.00280,53.0,0.00280,51,0.00280,49,0.00295,46.0,0.00295,41.0,59913,3,"{\n ""Year"": 2024,\n ""Make"": ""Acura"",\n ""Mod...",2024,Acura,Integra,Hatchback
9,2025 Acura Integra Hatchback,Integra,American Honda Finance (Acura),36195,34864,0.963227,0,34864,34864,NaN,NaN,Captive Lease Offer (350)\nAcura Lease Cash (0...,350,34514,DE4H3SJW 5dr HB A-Spec Package AT,0.00180,62.0,0.00180,61.0,0.00180,59.0,0.00180,57.0,0.00180,55,0.00180,53,0.00180,51.0,0.00180,47.0,72032,3,"{\n ""Year"": 2025,\n ""Make"": ""Acura"",\n ""Mod...",2025,Acura,Integra,Hatchback
10,2024 Acura Integra Hatchback,Integra,American Honda Finance (Acura),37695,36241,0.961427,-600,36841,36841,NaN,NaN,Acura Lease Cash (0)\nTotal: 0\nlink,0,36841,DE4H6RJW 5dr HB A-Spec Tech Package AT,0.00380,58.0,0.00380,57.0,0.00380,55.0,0.00280,53.0,0.00280,51,0.00280,49,0.00295,46.0,0.00295,41.0,59915,3,"{\n ""Year"": 2024,\n ""Make"": ""Acura"",\n ""Mod...",2024,Acura,Integra,Hatchback
12,2025 Acura Integra Hatchback,Integra,American Honda Finance (Acura),39195,37689,0.961577,0,37689,37689,NaN,NaN,Captive Lease Offer (350)\nAcura Lease Cash (0...,350,3

In [114]:
# months = int(input("Enter lease term in months (24, 27, 30, 33, 36, 39, 42, 48): "))
# bank = input("Enter bank name: ")
# package = input("Enter package name: ")
months = 24
bank = "American Honda Finance (Acura)"
package = "2024 Acura ZDX AWD SUV"
# Filter data based on the inputs
package_data = df[(df['Package'] == package) & (df['Bank'] == bank)]

if not package_data.empty:
    # Extract values from the selected row
    msrp = package_data['MSRP'].values[0]
    adjusted_cap_cost = package_data['Adjusted Cap Cost'].values[0]
    print(msrp,adjusted_cap_cost)
    
    # Residual Value Calculation
    rp_column = f'RP {months}'
    residual_value = msrp * package_data[rp_column].values[0] * 0.01
    
    # Money Factor Calculation
    mf_column = f'MF {months}'
    money_factor = package_data[mf_column].values[0]
    print(money_factor)
    
    # Depreciation Fee (Lease Cost)
    depreciation_fee = (adjusted_cap_cost - residual_value) / months
    print(depreciation_fee)
    
    # Finance Fee (Rent Charge)
    finance_fee = (adjusted_cap_cost + residual_value) * money_factor
    
    # Tax Rate and Taxes Calculation
    tax_rate = 8.875 / 100
    taxes = (depreciation_fee + finance_fee) * tax_rate
    
    # Total Monthly Lease Payment
    monthly_payment = depreciation_fee + finance_fee + taxes
    
    # Print the result
    print(f"Monthly Lease Payment: ${monthly_payment:.2f}")
else:
    print("No data found for the selected package and bank.")

70450 47358
0.0012
564.25
Monthly Lease Payment: $720.38


In [111]:
package_data

,Package,Rate Model,Bank,MSRP,Invoice,Formula,Buying,Selling Price_36,Selling Price,Default Rebates,Nondefault Rebates,Default Rebates_36,Total_Rebates,Adjusted Cap Cost,Rate Style,MF 24,RP 24,MF 27,RP 27,MF 30,RP 30,MF 33,RP 33,MF 36,RP 36,MF 39,RP 39,MF 42,RP 42,MF 48,RP 48,Package Id,Bank Id
0,2024 Acura ZDX AWD SUV,ZDX AWD,American Honda Finance (Acura),70450,69208,0.982370,0,69208,69208,NaN,NaN,"Clean Vehicle Lease Credit (7,500)\nCaptive Le...",21850,47358,3B2H4REW 4dr Wgn A-Spec,0.0012,48.0,0.0012,47.0,0.0012,45.0,0.0012,43.0,0.0012,41,0.0028,39,0.00295,36.0,0.00295,31.0,70563,3
1,2024 Acura ZDX AWD SUV,ZDX AWD,American Honda Finance (Acura),75450,73970,0.980384,0,73970,73970,NaN,NaN,"Clean Vehicle Lease Credit (7,500)\nCaptive Le...",18350,55620,3B2H6REW 4dr Wgn Type S,0.0012,47.0,0.0012,46.0,0.0012,44.0,0.0012,42.0,0.0012,40,0.0028,38,0.00295,35.0,0.00295,30.0,70564,3
